In [ ]:
// boring imports
var Plot = require('plotly-notebook-js');
var {loadLabelledWine} = require('./utils');

// Added some utility functions for arrays
// prep our input data structures
Array.prototype.max = function() {
  return Math.max.apply(null, this);
};

Array.prototype.min = function() {
  return Math.min.apply(null, this);
};

# Supervised Learning


![Supervised learning](images/slide_supervised.png)

### Techniques availble in mljs

 - KNN - K Nearest Neighbors
 - **SVM - Support Vector Machines**
 - Naive Bayes
 - Partial Least Squares [regression]



In [ ]:
var {features, dataset} = loadLabelledWine({verbose: true});

## Support Vector Machines


##### SVM in libsvm-js

 - docs for the SVM module are [here](https://github.com/mljs/libsvm)
 - docs for the various kernel options are [here](https://github.com/mljs/libsvm#SVM.KERNEL_TYPES)


In [ ]:
var SVM = require('libsvm-js/asm');

var options = {
    kernel: SVM.KERNEL_TYPES.RBF, // The type of kernel I want to use
    type: SVM.SVM_TYPES.C_SVC,    // The type of SVM I want to run
    gamma: 1,                     // RBF kernel gamma parameter
    cost: 1                       // C_SVC cost parameter
};

var svm = new SVM(options);

Next train the classifier, let's start with the original 2d feature vectors that we used in our first pass through kmeans.

To do this, just call the `svm.train(inputs, labels)` with the input feature vectors and the true class labels.

In [ ]:
var inputs = dataset.map(d => [d[0], d[10]]);
var labels = dataset.map(d => d[13]);

svm.train(inputs, labels)

console.log(labels.slice(50,70))
console.log(svm.predict(inputs.slice(50,70)))

#### Support Vectors

In [ ]:
console.log(svm.getSVIndices().slice(0,5))

#### Measure Accuracy

Use the same confusion matrix approach as earlier to compute accuracy and f1-scores

In [ ]:
var ConfusionMatrix = require('ml-confusion-matrix');

var actuals = labels;
var predicted = svm.predict(inputs)

var C = ConfusionMatrix.fromLabels(actuals, predicted)

var M = C.getMatrix();
var trace = { 
    x: [0,1,2],
    y: [0,1,2],
    z: M,
    type: 'heatmap',
    showscale: false,
    colorscale:[[0, '#3D9970'], [100, '#001f3f']]
};

console.info(C.getAccuracy())
var annotations = [];

M.map((a,y) => {
    a.map((b,x) => {
        annotations.push(
            {
                x: x,
                y: y,
                text: M[y][x],
                font: {
                    family: 'Arial',
                    size: 12,
                    color: 'white'
                  },
                showarrow: false
            }
        )
    })
})

var layout = { 
    xaxis: { title: "predicted", side: 'top' },
    yaxis: { title: "actuals", nticks: 6, autosize: false, autorange: 'reversed' },
    annotations,
    width: 500, height: 500};

var Plot = require('plotly-notebook-js');
var table = require('text-table');

$$html$$ = Plot.createPlot([trace], layout).render();

#### Discussion

 - How does you accuracy compare with our unsupervised approaches?
 - Is this too good to be tru and should you be suspicous?
 - Can you think of how to get a better accuracy measurement?